In [ ]:
%%bash
uw_trainging=/scratch3/BMC/wrfruc/cholt/uw_training_prep/uw_training
uw chgres_cube run --cycle 2025-08-27T12 --leadtime 0 -c $uw_training/uw_configs/prepare_bcs.yaml --batch --key-path make_ics

In [ ]:
%%bash
for i in $(seq 0 6) ; do uw chgres_cube run --cycle 2025-08-27T12 --leadtime $i -c $uw_training/uw_configs/prepare_bcs.yaml --batch --key-path make_lbcs ; done

In [ ]:
%%bash
uw fv3 run --cycle 2025-08-27T12 -c $uw_training/uw_configs/fv3_config.yaml --batch --key-path forecast_prod

In [ ]:
%%bash
uw gsi run --cycle 2025-08-27T13 -c $uw_training/uw_training/uw_configs/gsi_uw_config.yaml --key-path analysis --batch

In [ ]:
%%bash
$uw_training/scripts/run_forecast.sh $uw_training/uw_configs/fv3_config.yaml 2025-08-27T13